In [1]:
import numpy as np
import pandas as pd

In [68]:
# Read the data
df = pd.read_csv('uberdrive.csv')
df_miles = pd.read_csv('UberDrive_Miles.csv')

In [26]:
df.head(10)

,Trip_Id,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,PURPOSE*
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain
6,7,01-06-2016 17:30,01-06-2016 17:35,Business,West Palm Beach,Palm Beach,Meeting
7,8,01-07-2016 13:27,01-07-2016 13:33,Business,Cary,Cary,Meeting
8,9,01-10-2016 08:05,01-10-2016 08:25,Business,Cary,Morrisville,Meeting
9,10,01-10-2016 12:17,01-10-2016 12:44,Business,Jamaica,New York,Customer Visit


In [27]:
df_miles.head(5)

,Trip_Id,MILES*
0,1,5.1
1,2,5.0
2,3,4.8
3,4,4.7
4,5,63.7


# Notes from Josh
https://uoa-eresearch.github.io/eresearch-cookbook/recipe/2014/11/20/conda/
* Conda activate yourenvname

* conda create -n conda-env anaconda
* Conda activate condo-env
* Conda install numpy pandas
* Jupyter notebook
* conda info --envs

# Objective

Objective-
The aim is to create a driver profile based on the below aspects on driving behavior.

1. Name and number of all the unique start and stop points
2. Popular start and stop points
3. Rides with same start and stop points
4. Starting point from which most miles have been driven
5. Start- stop pairs that are most travelled in terms of distance
6. busiest month in terms of number of drives and miles driven
7. busiest day of the week
8. peak hours
9. most frequent trip category
10. most frequent trip purpose
11. miles driven per category and purpose
12. percent composition of business miles vs personal miles
13. time spent per category and purpose

In [28]:
# Getting the shape and size of dataframe

# shape - Gives the (R,C) of the dataframe
print("-----df shape ------------", df.shape)
print("-----df_miles shape------------", df_miles.shape)
print()

# size - Gives R * C value (Total values)
print("-----df size------------", df.size)
print("-----df_miles size------------", df_miles.size)
print()
print("**************************************")
# info - Gives the column names of the dataframe
print(df.info())
# Get the info for miles csv
print(df_miles.info())

-----df shape ------------ (1155, 7)
-----df_miles shape------------ (1155, 2)

-----df size------------ 8085
-----df_miles size------------ 2310

**************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Trip_Id      1155 non-null   int64 
 1   START_DATE*  1155 non-null   object
 2   END_DATE*    1155 non-null   object
 3   CATEGORY*    1155 non-null   object
 4   START*       1155 non-null   object
 5   STOP*        1155 non-null   object
 6   PURPOSE*     653 non-null    object
dtypes: int64(1), object(6)
memory usage: 63.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Trip_Id  1155 non-null   int64  
 1   MILES*   1155 non-null   float64
dtypes: float64(1), int64

In [71]:
# Renaming the cell - Approach 1
# Gets the column names
print(df.columns)

# Cleaning the column name to remove *
df.columns = df.columns.str.replace("*","")
print(df.columns)

# Renaming the cell - Approach 2
print(df_miles.columns)
df_miles.rename(columns = {"MILES*" : "MILES"}, inplace=True)
print(df_miles.columns)

Index(['Trip_Id', 'START_DATE*', 'END_DATE*', 'CATEGORY*', 'START*', 'STOP*',
       'PURPOSE*', 'MILES*'],
      dtype='object')
Index(['Trip_Id', 'START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP',
       'PURPOSE', 'MILES'],
      dtype='object')
Index(['Trip_Id', 'MILES*'], dtype='object')
Index(['Trip_Id', 'MILES'], dtype='object')


In [36]:
# 1. Name and number of all the unique start and stop points
# To get unique values for a series use .unique()
print(df['START'].unique())
# Get the unique value count of each value
print("------- NUMBER OF UNIQUE START POINTS -------->>",df['START'].nunique()) 
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%");
print()
print(df['STOP'].unique())
print("------- NUMBER OF UNIQUE STOP POINTS -------->>",len(df['STOP'].unique()))
print()

['Fort Pierce' 'West Palm Beach' 'Cary' 'Jamaica' 'New York' 'Elmhurst'
 'Midtown' 'East Harlem' 'Flatiron District' 'Midtown East'
 'Hudson Square' 'Lower Manhattan' "Hell's Kitchen" 'Downtown' 'Gulfton'
 'Houston' 'Eagan Park' 'Morrisville' 'Durham' 'Farmington Woods'
 'Whitebridge' 'Lake Wellingborough' 'Fayetteville Street' 'Raleigh'
 'Hazelwood' 'Fairmont' 'Meredith Townes' 'Apex' 'Chapel Hill'
 'Northwoods' 'Edgehill Farms' 'Tanglewood' 'Preston' 'Eastgate'
 'East Elmhurst' 'Jackson Heights' 'Long Island City' 'Katunayaka'
 'Unknown Location' 'Colombo' 'Nugegoda' 'Islamabad' 'R?walpindi'
 'Noorpur Shahan' 'Heritage Pines' 'Westpark Place' 'Waverly Place'
 'Wayne Ridge' 'Weston' 'East Austin' 'West University' 'South Congress'
 'The Drag' 'Congress Ave District' 'Red River District' 'Georgian Acres'
 'North Austin' 'Coxville' 'Convention Center District' 'Austin' 'Katy'
 'Sharpstown' 'Sugar Land' 'Galveston' 'Port Bolivar' 'Washington Avenue'
 'Briar Meadow' 'Latta' 'Jacksonville'

In [40]:
# 2. Popular start and stop points
# 2a. Popular start points
print("-----POPULAR START POINTS------")
print(df['START'].value_counts().head(10))
print()
# 2b. Popular stop points
print("-----POPULAR STOP POINTS------")
print(df['STOP'].value_counts().head(10))
print()

-----POPULAR START POINTS------
Cary                201
Unknown Location    148
Morrisville          85
Whitebridge          68
Islamabad            57
Durham               37
Lahore               36
Raleigh              28
Kar?chi              27
Westpark Place       17
Name: START, dtype: int64

-----POPULAR STOP POINTS------
Cary                203
Unknown Location    149
Morrisville          84
Whitebridge          65
Islamabad            58
Lahore               36
Durham               36
Raleigh              29
Kar?chi              26
Apex                 17
Name: STOP, dtype: int64



In [45]:
# 3. Rides with same start and stop points
# Gets all the details where start and stop is the same
df[df['START'] == df['STOP']]

# Gets the [R * C] by using shape
df[df['START'] == df['STOP']].shape

(288, 7)

In [70]:
# 4. Starting point from which most miles have been driven
# We do a join on the two csvs on 'Trip_ID' -> outer join and get the top 5 values
df.columns
df = pd.merge(df, df_miles, on = 'Trip_Id', how = 'left')
df.head(5)

,Trip_Id,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,PURPOSE*,MILES*
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7


In [72]:
# 5. Start- stop pairs that are most travelled in terms of distance
df.groupby('START')['MILES'].sum().sort_values(ascending = False ).head(10)

START
Unknown Location    1976.5
Cary                1791.3
Morrisville          671.7
Raleigh              433.0
Islamabad            401.2
Durham               384.4
Jacksonville         375.2
Latta                310.3
Asheville            287.7
Whitebridge          273.4
Name: MILES, dtype: float64